In [64]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
import matplotlib.pyplot as plt
import json
import timeit

In [2]:
import numpy as np
import requests
from requests.auth import HTTPBasicAuth

from planet import api
from planet.api import filters
from sys import stdout
import io
import tempfile

from datetime import date, datetime, timedelta
import time
import os
import pathlib
import fnmatch

In [74]:
latitude_1 = input("Input Latitude: ")
latitude_1 = float(latitude_1)

longitude_1 = input("Input Longitude: ")
longitude_1 = float(longitude_1)

# Taking lat and long into pandas dataframe
point_df = pd.DataFrame({'latitude':[latitude_1],
                     'longitude':[longitude_1]})

# Converting pandas DF to geopandas GDF
point_gdf = gpd.GeoDataFrame(point_df, geometry=gpd.points_from_xy(point_df.longitude, point_df.latitude))




# Converting geodataframe to Meters from Lat/Long
# Allows for square buffer to be applied (450m)

point_gdf.crs = {'init' :'epsg:4326'} 
point_gdf_m = point_gdf.to_crs(epsg=3395)

# Applying the buffer, cap_style = 3 --> Square Buffer
buffer = point_gdf_m.buffer(450, cap_style=3)


# Convert buffer to wgs84
buffer_wgs84 = buffer.to_crs(epsg=4326)
# Convert to json
buffer_wgs84_json = buffer_wgs84.to_json()
# Loading and printing the json
buffer_wgs84_json_parsed = json.loads(buffer_wgs84_json)
buffer_wgs84_json_api = buffer_wgs84_json_parsed['features'][0]['geometry']['coordinates']
# print(json.dumps(buffer_wgs84_json_parsed, indent=2, sort_keys=True))


geojson_geometry = {
    "type": "Point",
    "coordinates": [
         longitude_1,latitude_1
        ]
}

# get images that overlap with our AOI 
geometry_filter = {
  "type": "GeometryFilter",
  "field_name": "geometry",
  "config": geojson_geometry
}

# Generating time parameters
today = datetime.isoformat(datetime.utcnow())+'Z'#(datetime.today())
start_date = datetime.isoformat(datetime.utcnow() - timedelta(7)) + 'Z'

# get images acquired within a date range
date_range_filter = {
  "type": "DateRangeFilter",
  "field_name": "acquired",
  "config": {
    "gte": start_date,
    "lte": today
  }
}

# only get images which have <10% cloud coverage
cloud_cover_filter = {
  "type": "RangeFilter",
  "field_name": "cloud_cover",
  "config": {
    "lte": 0.1
  }
}

# combine our geo, date, cloud filters
combined_filter = {
  "type": "AndFilter",
  "config": [geometry_filter, date_range_filter, cloud_cover_filter]
}



# Generating Order
# API Key stored as an env variable
PL_API_KEY = 'XXXXXXXXXXXXXX'
PLANET_API_KEY = PL_API_KEY #os.getenv('PL_API_KEY')
orders_url = 'https://api.planet.com/compute/ops/orders/v2'

# set up requests to work with api
auth = HTTPBasicAuth(PLANET_API_KEY, '')
headers = {'content-type': 'application/json'}

item_type = "PSScene4Band"

# API request object
search_request = {
    "interval": "day",
    "item_types": [item_type],
    "asset_types" : "analytic_sr",
    "filter": combined_filter
}

# fire off the POST request
search_result = \
  requests.post(
    'https://api.planet.com/data/v1/quick-search',
#     'https://api.planet.com/data/v2',
    auth=HTTPBasicAuth(PLANET_API_KEY, ''),
    json=search_request)


# extract image IDs only
image_ids = [feature['id'] for feature in search_result.json()['features']]

# For demo purposes, just grab the first image ID
id0 = image_ids[0]
id0_url = 'https://api.planet.com/data/v1/item-types/{}/items/{}/assets'.format(item_type, id0)

# Returns JSON metadata for assets in this ID. Learn more: planet.com/docs/reference/data-api/items-assets/#asset
result = \
  requests.get(
    id0_url,
    auth=HTTPBasicAuth(PLANET_API_KEY, '')
  )


# Parse out useful links
links = result.json()[u"analytic_sr"]["_links"]
self_link = links["_self"]
activation_link = links["activate"]

# Request activation of the 'visual' asset:
activate_result = \
  requests.get(
    activation_link,
    auth=HTTPBasicAuth(PLANET_API_KEY, '')
  )




# define products part of order
single_product = [
    {
      "item_ids": [id0],#["20151119_025740_0c74"],
      "item_type": "PSScene4Band",
      "product_bundle": "analytic_sr"
    }
]



# define helpful functions for submitting, polling, and downloading an order
def place_order(request, auth):
    response = requests.post(orders_url, data=json.dumps(request), auth=auth, headers=headers)
    print(response)
    
    if not response.ok:
        raise Exception(response.content)

    order_id = response.json()['id']
    print(order_id)
    order_url = orders_url + '/' + order_id
    return order_url

def poll_for_success(order_url, auth, num_loops=50):
    count = 0
    while(count < num_loops):
        count += 1
        r = requests.get(order_url, auth=auth)
        response = r.json()
        state = response['state']
        print(state)
        success_states = ['success', 'partial']
        if state == 'failed':
            raise Exception(response)
        elif state in success_states:
            break
        
        time.sleep(10)
        
def download_order(order_url, auth, overwrite=False):
    r = requests.get(order_url, auth=auth)
    print(r)

    response = r.json()
    results = response['_links']['results']
    results_urls = [r['location'] for r in results]
    results_names = [r['name'] for r in results]
    results_paths = [pathlib.Path(os.path.join('data', n)) for n in results_names]
    print('{} items to download'.format(len(results_urls)))
    
    for url, name, path in zip(results_urls, results_names, results_paths):
        if overwrite or not path.exists():
            print('downloading {} to {}'.format(name, path))
            r = requests.get(url, allow_redirects=True)
            path.parent.mkdir(parents=True, exist_ok=True)
            open(path, 'wb').write(r.content)
        else:
            print('{} already exists, skipping {}'.format(path, name))
            
    return dict(zip(results_names, results_paths))


def place_order(request, auth):
    response = requests.post(orders_url, data=json.dumps(request), auth=auth, headers=headers)
    print(response)
    order_id = response.json()['id']
    print(order_id)
    order_url = orders_url + '/' + order_id
    return order_url


clip_aoi = {
    "type":"Polygon",
    "coordinates":buffer_wgs84_json_api       #buffer_wgs84_json_parsed['features'][0]['geometry']['coordinates']
}

# define the clip tool
clip = {
    "clip": {
        "aoi": clip_aoi
    }
}

# create an order request with the clipping tool
request_clip = {
  "name": "just clip",
  "products": single_product,
  "tools": [clip]
}





# create the order URL
order_url = place_order(request_clip, auth)


# # allow for caching so we don't always run clip
# run_clip = True

# if run_clip:
#     clip_order_url = place_order(request_clip, auth)
#     poll_for_success(clip_order_url, auth)



# # Download the order
# download_order(clip_order_url, auth)




Input Latitude: 39.577762
Input Longitude: -106.092863


/opt/anaconda3/lib/python3.7/site-packages/pyproj/crs.py:77: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method.
  return _prepare_from_string(" ".join(pjargs))
/opt/anaconda3/lib/python3.7/site-packages/pyproj/crs.py:77: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method.
  return _prepare_from_string(" ".join(pjargs))


<Response [202]>
cbb43520-0a0d-4c78-8ece-d1cff28fe9d1


In [75]:
# allow for caching so we don't always run clip
run_clip = True

if run_clip:
    clip_order_url = place_order(request_clip, auth)
    poll_for_success(clip_order_url, auth)



# Download the order
download_order(clip_order_url, auth)




<Response [202]>
e19a4432-5e2c-4ced-b9e9-3d25786c55a1
queued
queued
queued
queued
queued
queued
queued
queued
queued
queued
queued
queued
queued
running
running
running
running
running
running
running
running
running
running
running
success
<Response [200]>
5 items to download
downloading e19a4432-5e2c-4ced-b9e9-3d25786c55a1/1/manifest.json to data/e19a4432-5e2c-4ced-b9e9-3d25786c55a1/1/manifest.json
downloading e19a4432-5e2c-4ced-b9e9-3d25786c55a1/1/files/20200303_154531_0f1a_3B_AnalyticMS_DN_udm_clip.tif to data/e19a4432-5e2c-4ced-b9e9-3d25786c55a1/1/files/20200303_154531_0f1a_3B_AnalyticMS_DN_udm_clip.tif
downloading e19a4432-5e2c-4ced-b9e9-3d25786c55a1/1/files/20200303_154531_0f1a_metadata.json to data/e19a4432-5e2c-4ced-b9e9-3d25786c55a1/1/files/20200303_154531_0f1a_metadata.json
downloading e19a4432-5e2c-4ced-b9e9-3d25786c55a1/1/files/20200303_154531_0f1a_3B_AnalyticMS_SR_clip.tif to data/e19a4432-5e2c-4ced-b9e9-3d25786c55a1/1/files/20200303_154531_0f1a_3B_AnalyticMS_SR_clip.tif


{'e19a4432-5e2c-4ced-b9e9-3d25786c55a1/1/manifest.json': PosixPath('data/e19a4432-5e2c-4ced-b9e9-3d25786c55a1/1/manifest.json'),
 'e19a4432-5e2c-4ced-b9e9-3d25786c55a1/1/files/20200303_154531_0f1a_3B_AnalyticMS_DN_udm_clip.tif': PosixPath('data/e19a4432-5e2c-4ced-b9e9-3d25786c55a1/1/files/20200303_154531_0f1a_3B_AnalyticMS_DN_udm_clip.tif'),
 'e19a4432-5e2c-4ced-b9e9-3d25786c55a1/1/files/20200303_154531_0f1a_metadata.json': PosixPath('data/e19a4432-5e2c-4ced-b9e9-3d25786c55a1/1/files/20200303_154531_0f1a_metadata.json'),
 'e19a4432-5e2c-4ced-b9e9-3d25786c55a1/1/files/20200303_154531_0f1a_3B_AnalyticMS_SR_clip.tif': PosixPath('data/e19a4432-5e2c-4ced-b9e9-3d25786c55a1/1/files/20200303_154531_0f1a_3B_AnalyticMS_SR_clip.tif'),
 'e19a4432-5e2c-4ced-b9e9-3d25786c55a1/1/files/20200303_154531_0f1a_3B_AnalyticMS_metadata_clip.xml': PosixPath('data/e19a4432-5e2c-4ced-b9e9-3d25786c55a1/1/files/20200303_154531_0f1a_3B_AnalyticMS_metadata_clip.xml')}

In [ ]:
# 39.479679, -106.046778
# 39.577762, -106.092863

In [ ]:
# Input Latitude: 32.74299535682746
# Input Longitude: -116.46464824676514